In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn import preprocessing

In [2]:
# Import dataset with panda
df = pd.read_csv('output.csv')
features_considered = ['embb_req', 'embb_true', 'urllc_req', 'urllc_true','miot_req', 'miot_true','mmtc_req', 'mmtc_true','voice_req', 'voice_true']
features = df[features_considered]
features.index = df['round']
features.head()


,embb_req,embb_true,urllc_req,urllc_true,miot_req,miot_true,mmtc_req,mmtc_true,voice_req,voice_true
round,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0
1,14135738986,14135738986,183636064,183636064,132047432,132047432,174948154,174948154,256599799,256599799
2,41280858627,17850282270,329891717,329891717,285377529,285377529,369429583,369429583,649253798,632765102
3,38436413381,18163677631,321798499,321798499,183000490,183000490,211807759,211807759,617741942,617325258
4,35012592777,17970187108,283394642,283394642,318840447,318840447,388482214,388482214,554585090,553071014


In [3]:
# Parametes 
TRAIN_SPLIT = 64000
tf.random.set_seed(13)
past_history = 1
future_target = 0
step = 1
BATCH_SIZE = 32 # combines consecutive elements of this dataset into batches.
BUFFER_SIZE = 80000 # >= len dataset for perfetct shuffling
EVALUATION_INTERVAL = 2000 # steps_per_epoch = TotalTrainingSamples / TrainingBatchSize
VALIDATION_STEPS = 500 # validation_steps = TotalvalidationSamples / ValidationBatchSize
EPOCHS = 20
bs_capacity = 20000000000


In [4]:
# Normalize
dataset = features.values
dataset = dataset / bs_capacity

In [5]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)

    data.append(dataset[indices])
    labels.append(target[i+target_size])

  return np.array(data), np.array(labels)


In [6]:
# Create validation and training set
x_train, y_train = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], 0, TRAIN_SPLIT, past_history,future_target, step)
x_val, y_val = multivariate_data(dataset[1:, (0, 2, 4, 6, 8)], dataset[1:, (1, 3, 5, 7, 9)], TRAIN_SPLIT, None, past_history, future_target, step)

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.batch(BATCH_SIZE).repeat()


In [7]:
# Create RNN
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=x_train.shape[-2:])) # 4*(len features input * len output + input^2)
model.add(tf.keras.layers.Dense(5))

# Compile RNN
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

# Train RNN
history = model.fit(train_data, epochs=EPOCHS, steps_per_epoch=EVALUATION_INTERVAL, validation_data=val_data, validation_steps=VALIDATION_STEPS)


Train for 2000 steps, validate for 500 steps
Epoch 1/20
2000/2000 [==============================] - 13s 6ms/step - loss: 0.0269 - val_loss: 0.0287
Epoch 2/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0236 - val_loss: 0.0276
Epoch 3/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0232 - val_loss: 0.0287
Epoch 4/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0229 - val_loss: 0.0277
Epoch 5/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0228 - val_loss: 0.0268
Epoch 6/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0226 - val_loss: 0.0272
Epoch 7/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0225 - val_loss: 0.0280
Epoch 8/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0224 - val_loss: 0.0268
Epoch 9/20
2000/2000 [==============================] - 11s 5ms/step - loss: 0.0223 - val_loss: 0.0275
Epoch 10/20
2000/2000 [=====

In [8]:
#Example prediction
prediction = model.predict(x_val, batch_size=BATCH_SIZE)[0]

print(prediction)
print(y_val[0])

# Prediction with new input value
test = [[[29134251061,187609472,194216024,110492056,439310873]]]
test = np.array(test)/bs_capacity

prediction = model.predict(test)
print(prediction[0])
print(y_train[2]) # because I took the input of the first values in x_train for test


[0.88226527 0.01002413 0.00731078 0.0102368  0.01273997]
[0.76252162 0.00797926 0.00862973 0.00942115 0.00949748]
[0.9038942  0.00976555 0.00725922 0.00992475 0.01576033]
[0.89850936 0.01416973 0.01594202 0.01942411 0.02765355]


In [9]:
# Save trained model
model.save("model.h5")
